##fMRI Analysis
1. **preprocess**
2. Individual Level analysis
3. Group Level analysis


###preprocess
1. slice time correction(fsl)
2. motion correction (fsl)
3. artifact detection to identify outliers (fsl)
4. segentation - fsl FAST
5. coregistration with anatomical (fsl)
6. smoothing (fsl)

In [ ]:
!pip install nipype
!pip install nilearn
!pip install bids

##Imports

In [ ]:
%matplotlib inline
from os.path import join as opj
import os
import json
from nipype.interfaces.fsl import (BET, ExtractROI, FAST, FLIRT, ImageMaths, MCFLIRT, SliceTimer, Threshold)
from nipype.interfaces.spm import Smooth
from nipype.interfaces.utility import IdentityInterface
from nipype.interfaces.io import SelectFiles, DataSink
from nipype.algorithms.rapidart import ArtifactDetect
from nipype import Workflow, Node

In [ ]:
path = os.listdir('/content/fmri')
path

['sub-xp101-anat-sub-xp101_T1w.nii.gz',
 'sub-xp101-func-sub-xp101_task-fmriNF_bold.nii.gz',
 'sub-xp101-anat-sub-xp101_T1w.json',
 'sub-xp101-func-sub-xp101_task-motorloc_bold.json',
 'sub-xp101-func-sub-xp101_task-motorloc_bold.nii.gz']

##Single subject run

In [ ]:
from nipype import Node, Workflow

In [ ]:
from nipype.interfaces.fsl import SliceTimer, MCFLIRT, Smooth
slicetimer = Node(SliceTimer(index_dir=False,
                             interleaved=True, 
                             time_repetition=2.5),
                             name="slicetimer")

In [ ]:
mcflirt = Node(MCFLIRT(mean_vol=True,
                       save_plots=True),
               name="mcflirt")

In [ ]:
smooth = Node(Smooth(fwhm=4), name="smooth")

In [ ]:
preproc01 = Workflow(name='preproc01', base_dir='.')

In [ ]:
preproc01.connect([(slicetimer, mcflirt, 
                   [('slice_time_corrected_file', 'in_file')]),
                   (mcflirt, smooth,
                    [('out_file', 'in_file')])])

In [ ]:
preproc01.write_graph(graph2use='orig')

210209-19:44:09,638 nipype.workflow INFO:
	 Generated workflow graph: /content/preproc01/graph.png (graph2use=orig, simple_form=True).


'/content/preproc01/graph.png'

In [ ]:
from IPython.display import Image
Image(filename = "/content/preproc01/graph_detailed.png")

In [ ]:
slicetimer.inputs.in_file = '/content/fmri/sub-xp101-func-sub-xp101_task-motorloc_bold.nii.gz'

In [ ]:
preproc01.run()